In [ ]:
# ===================================================================
# SECTION 9: TEORI PELUANG DISKRIT (DISCRETE PROBABILITY)
# ===================================================================

print("\n" + "="*90)
print(" 9. TEORI PELUANG DISKRIT (DISCRETE PROBABILITY) ".center(90))
print("="*90)

print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                     KONSEP MATEMATIKA DISKRIT                                ║
╚══════════════════════════════════════════════════════════════════════════════╝

Teori peluang untuk ruang sampel yang diskrit (finite atau countable).

KONSEP DASAR:

  Sample Space S: S = {s₁, s₂, ..., sₙ}
  Probabilitas: P(A) = |A| / |S|

  Aksioma Kolmogorov:
  1. P(A) ≥ 0
  2. P(S) = 1
  3. P(A ∪ B) = P(A) + P(B) jika A ∩ B = Ø

PROBABILITY MASS FUNCTION (PMF):
  PMF: P(X = xᵢ) untuk setiap nilai xᵢ
  Properti: Σ P(X = xᵢ) = 1

DISCRETE EXPECTED VALUE:
  E(X) = Σ xᵢ × P(X = xᵢ)

MARKOV CHAINS:
  Transisi multi-step: Pⁿ = P × P × ... × P (n kali)
""")

print(f"\nPROBABILITY MASS FUNCTION (PMF):")
print(f"  Random variable X = persentase kenaikan harga")

max_increase = df_all['increase_pct'].max()
min_increase = df_all['increase_pct'].min()

if max_increase < 30:
    bins = [float('-inf'), 0, 5, 10, 15, 20, float('inf')]
    labels = ['<0%', '0-5%', '5-10%', '10-15%', '15-20%', '>20%']
    midpoints = {'<0%': min_increase/2, '0-5%': 2.5, '5-10%': 7.5,
                 '10-15%': 12.5, '15-20%': 17.5, '>20%': (max_increase+20)/2}
else:
    bins = [float('-inf'), 0, 25, 50, 75, 100, float('inf')]
    labels = ['<0%', '0-25%', '25-50%', '50-75%', '75-100%', '>100%']
    midpoints = {'<0%': min_increase/2, '0-25%': 12.5, '25-50%': 37.5,
                 '50-75%': 62.5, '75-100%': 87.5, '>100%': (max_increase+100)/2}

df_all['increase_category'] = pd.cut(df_all['increase_pct'], bins=bins, labels=labels)
pmf = df_all['increase_category'].value_counts(normalize=True).sort_index()

print(f"\n  Distribusi probabilitas:")
for category, prob in pmf.items():
    if prob > 0.01:
        bar = '█' * int(prob * 40)
        print(f"    P(X ∈ {str(category):10s}) = {prob:.4f}  {bar}")

print(f"\n  ✓ Σ P(X = xᵢ) = {pmf.sum():.4f} (Aksioma Kolmogorov)")

expected_value = sum(midpoints[cat] * prob for cat, prob in pmf.items() if cat in midpoints)

print(f"\nDISCRETE EXPECTED VALUE:")
print(f"  E(X) = {expected_value:.2f}%")

if expected_value < 0:
    print(f"  Interpretasi: Baseline = minggu awal, jadi rata-rata di bawah baseline (wajar)")
elif expected_value < 5:
    print(f"  Interpretasi: Ekspektasi kenaikan rendah (stabil)")
else:
    print(f"  Interpretasi: Ekspektasi kenaikan signifikan (volatile)")

print(f"\nMARKOV CHAINS:")

P = transition_prob.values
P2 = np.linalg.matrix_power(P, 2)
P3 = np.linalg.matrix_power(P, 3)

df_P2 = pd.DataFrame(P2, index=states, columns=states)
df_P3 = pd.DataFrame(P3, index=states, columns=states)

if 'Normal' in states and 'PUNCAK' in states:
    normal_idx = states.index('Normal')
    peak_idx = states.index('PUNCAK')

    prob_1 = P[normal_idx, peak_idx]
    prob_2 = P2[normal_idx, peak_idx]
    prob_3 = P3[normal_idx, peak_idx]

    print(f"  Probabilitas Normal → PUNCAK:")
    print(f"    1-step: {prob_1:.4f} ({prob_1*100:.2f}%)")
    print(f"    2-step: {prob_2:.4f} ({prob_2*100:.2f}%)")
    print(f"    3-step: {prob_3:.4f} ({prob_3*100:.2f}%)")

print(f"\n  Steady-state distribution:")
try:
    eigenvalues, eigenvectors = np.linalg.eig(P.T)
    stationary_idx = np.argmin(np.abs(eigenvalues - 1))
    stationary = np.real(eigenvectors[:, stationary_idx])
    stationary = stationary / stationary.sum()

    dominant_state = states[np.argmax(stationary)]
    dominant_prob = stationary.max()
    print(f"    State dominan jangka panjang: {dominant_state} ({dominant_prob*100:.1f}%)")
except:
    print(f"    Tidak dapat menghitung")

# ============================================================
# APLIKASI DUNIA NYATA
# ============================================================

print("\n" + "="*80)
print("╔══════════════════════════════════════════════════════════════════════════════╗")
print("║                          APLIKASI DUNIA NYATA                                ║")
print("╚══════════════════════════════════════════════════════════════════════════════╝")

print(f"""
PMF untuk distribusi probabilitas kenaikan harga dan prediksi multi-step dengan Markov Chains.

Manfaat: Estimasi expected return E(X) = {expected_value:.2f}% untuk planning profit margin.
         Pedagang dapat hitung probabilitas kenaikan dalam 2-3 minggu (P², P³).
         Manajemen risiko probabilistik: fokus pada range dengan P(X) tertinggi,
         hindari stok berlebih di range dengan probabilitas rendah.
""")

if SHOW_EXAMPLE:
    print(f"\n📊 STUDI KASUS: SEMUA KOMODITAS")
    print(f"{'─'*80}")
    print(f"\n(Berdasarkan expected value - Metrik Teori Peluang)")
    print(f"Analisis lengkap untuk semua {len(commodities_list)} komoditas tersedia di PMF di atas.")

    commodity_ev = []

    for commodity in commodities_list:
        comm_data = df_all[df_all['commodity'] == commodity].copy()

        comm_pmf = comm_data['increase_category'].value_counts(normalize=True).sort_index()

        comm_ev_val = sum(midpoints.get(cat, 0) * prob
                          for cat, prob in comm_pmf.items()
                          if cat in midpoints)

        commodity_ev.append({
            'commodity': commodity,
            'expected_value': comm_ev_val,
            'observations': len(comm_data)
        })

    ev_df = pd.DataFrame(commodity_ev).sort_values('expected_value', ascending=False)

    if len(ev_df) >= 2:
        highest_ev = ev_df.iloc[0]
        lowest_ev = ev_df.iloc[-1]

        print(f"\n  {highest_ev['commodity']} (Expected Value Tertinggi):")
        print(f"    E(X) = {highest_ev['expected_value']:.2f}%")
        print(f"    Observasi: {highest_ev['observations']}")
        print(f"    Interpretasi: Rata-rata kenaikan tertinggi dari baseline")

        print(f"\n  {lowest_ev['commodity']} (Expected Value Terendah):")
        print(f"    E(X) = {lowest_ev['expected_value']:.2f}%")
        print(f"    Observasi: {lowest_ev['observations']}")
        print(f"    Interpretasi: Paling stabil relatif terhadap baseline")

        print(f"\n{'─'*80}")
        print(f"PENJELASAN:")
        print(f"  → E(X) tinggi = komoditas cenderung naik jauh dari harga baseline")
        print(f"  → E(X) rendah = harga relatif stabil di sekitar baseline")
        print(f"  → PMF per komoditas berbeda, menunjukkan variasi pola volatilitas")
        print(f"  → Markov chains memberikan prediksi probabilistik 2-3 step ke depan")

else:
    print(f"\n📊 STUDI KASUS: {COMMODITY_GROUPS[kategori_choice]['name'].upper()}")
    print(f"{'─'*80}")
    print(f"\nExpected value untuk {len(commodities_list)} komoditas:")

    for commodity in commodities_list:
        comm_data = df_all[df_all['commodity'] == commodity].copy()

        comm_pmf = comm_data['increase_category'].value_counts(normalize=True).sort_index()

        comm_ev_val = sum(midpoints.get(cat, 0) * prob
                          for cat, prob in comm_pmf.items()
                          if cat in midpoints)

        print(f"\n  {commodity}:")
        print(f"    E(X) = {comm_ev_val:.2f}%")

        if len(comm_pmf) > 0:
            dominant_cat = comm_pmf.idxmax()
            dominant_prob = comm_pmf.max()
            print(f"    Kategori dominan: {dominant_cat} (P = {dominant_prob:.2f})")

    print(f"\n{'─'*80}")
    print(f"INSIGHT KATEGORI:")

    cat_data = df_all[df_all['commodity'].isin(commodities_list)]
    cat_pmf = cat_data['increase_category'].value_counts(normalize=True).sort_index()
    cat_ev = sum(midpoints.get(cat, 0) * prob
                 for cat, prob in cat_pmf.items()
                 if cat in midpoints)

    print(f"  → Kategori {COMMODITY_GROUPS[kategori_choice]['name']}: E(X) = {cat_ev:.2f}%")

    if cat_ev > expected_value:
        print(f"  → Expected value kategori LEBIH TINGGI dari global ({expected_value:.2f}%)")
    else:
        print(f"  → Expected value kategori LEBIH RENDAH dari global ({expected_value:.2f}%)")

print("\n" + "="*80)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].bar(range(len(pmf)), pmf.values,
               color='steelblue', alpha=0.8, edgecolor='black', linewidth=2)
axes[0, 0].set_xticks(range(len(pmf)))
axes[0, 0].set_xticklabels(pmf.index, rotation=45, ha='right')
axes[0, 0].set_ylabel('P(X = xᵢ)', fontsize=11, fontweight='bold')
axes[0, 0].set_title('PMF: Distribusi Probabilitas Diskrit', fontsize=12, fontweight='bold')
axes[0, 0].grid(True, alpha=0.3, axis='y')

for i, prob in enumerate(pmf.values):
    axes[0, 0].text(i, prob + 0.01, f'{prob:.3f}', ha='center', fontsize=9, fontweight='bold')

sns.heatmap(df_P2, annot=True, fmt='.3f', cmap='YlOrRd', ax=axes[0, 1],
            cbar_kws={'label': 'Probabilitas'}, linewidths=2, linecolor='black',
            annot_kws={'fontsize': 9, 'fontweight': 'bold'})
axes[0, 1].set_title('Markov: Transisi 2-Step (P²)', fontsize=12, fontweight='bold')
axes[0, 1].set_xlabel('State Tujuan', fontsize=10, fontweight='bold')
axes[0, 1].set_ylabel('State Awal', fontsize=10, fontweight='bold')

sns.heatmap(df_P3, annot=True, fmt='.3f', cmap='YlOrRd', ax=axes[1, 0],
            cbar_kws={'label': 'Probabilitas'}, linewidths=2, linecolor='black',
            annot_kws={'fontsize': 9, 'fontweight': 'bold'})
axes[1, 0].set_title('Markov: Transisi 3-Step (P³)', fontsize=12, fontweight='bold')
axes[1, 0].set_xlabel('State Tujuan', fontsize=10, fontweight='bold')
axes[1, 0].set_ylabel('State Awal', fontsize=10, fontweight='bold')

if 'Normal' in states:
    normal_idx = states.index('Normal')
    p1_from_normal = P[normal_idx, :]
    p2_from_normal = P2[normal_idx, :]
    p3_from_normal = P3[normal_idx, :]

    x = np.arange(len(states))
    width = 0.25

    axes[1, 1].bar(x - width, p1_from_normal, width, label='P¹', alpha=0.8, edgecolor='black')
    axes[1, 1].bar(x, p2_from_normal, width, label='P²', alpha=0.8, edgecolor='black')
    axes[1, 1].bar(x + width, p3_from_normal, width, label='P³', alpha=0.8, edgecolor='black')

    axes[1, 1].set_xlabel('State Tujuan', fontsize=11, fontweight='bold')
    axes[1, 1].set_ylabel('Probabilitas', fontsize=11, fontweight='bold')
    axes[1, 1].set_title('Evolusi Probabilitas dari Normal', fontsize=12, fontweight='bold')
    axes[1, 1].set_xticks(x)
    axes[1, 1].set_xticklabels(states, rotation=45, ha='right')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"\nKESIMPULAN:")
print(f"  Teori peluang diskrit mengidentifikasi:")
print(f"    • E(X) = {expected_value:.2f}% (rata-rata dari baseline minggu awal)")
if expected_value < 0:
    print(f"    • {pmf.iloc[0]*100:.1f}% observasi di bawah baseline (periode awal)")
    print(f"    • {pmf.iloc[1]*100:.1f}% observasi naik 0-25% (menjelang event)")
print(f"    • Markov: P³(Normal→PUNCAK) = {P3[states.index('Normal'), states.index('PUNCAK')]:.4f}")
print(f"    • PMF memvalidasi Σ P(X=xᵢ) = 1 (Aksioma Kolmogorov) ✓")

pause_section()

In [ ]:
# ===================================================================
# SECTION 10: DASHBOARD KOMPREHENSIF - DIPERBAIKI
# ===================================================================

print("\n" + "="*90)
print(" 10. DASHBOARD KOMPREHENSIF ".center(90))
print("="*90)

print("""
DASHBOARD VISUALISASI - INTEGRASI 9 KONSEP MATEMATIKA DISKRIT:

  1. Recurrence Relations      6. Kombinatorika
  2. Finite State Machine      7. Teori Bilangan
  3. Induksi Matematis         8. Teori Himpunan
  4. Teori Graf                9. Probabilitas Diskrit
  5. Boolean Logic

Total: 11 visualisasi dalam satu dashboard
""")


if 'intersection' not in locals() or 'union' not in locals():
    spike_threshold_set = df_all.groupby('commodity')['increase_pct'].quantile(0.75).mean()
    event_sets_local = {}
    for event in events:
        event_spikes = set()
        for commodity in commodities_list:
            mask = (df_all['commodity'] == commodity) & (df_all['event'] == event)
            if df_all[mask]['increase_pct'].max() >= spike_threshold_set:
                event_spikes.add(commodity)
        event_sets_local[event] = event_spikes

    intersection = set.intersection(*event_sets_local.values()) if event_sets_local else set()
    union = set.union(*event_sets_local.values()) if event_sets_local else set()
    complement = set(commodities_list) - union

always_spike = intersection
sometimes_spike = union
never_spike = complement


if 'expected_value' not in locals():
    if 'pmf' in locals():
        midpoints_temp = {'<0%': df_all['increase_pct'].min()/2}
        for cat in pmf.index[1:]:
            midpoints_temp[str(cat)] = df_all['increase_pct'].mean()
        expected_value = sum(midpoints_temp.get(str(cat), 0) * prob for cat, prob in pmf.items())
    else:
        expected_value = df_all['increase_pct'].mean()

fig = plt.figure(figsize=(20, 25))

fig.suptitle('DASHBOARD KOMPREHENSIF: 9 KONSEP MATEMATIKA DISKRIT\n' +
             'Analisis Pola Harga Barang Menjelang Hari Besar',
             fontsize=16, fontweight='bold', y=0.98)

gs = fig.add_gridspec(4, 3, hspace=0.35, wspace=0.30,
                      top=0.92, bottom=0.02, left=0.05, right=0.97)

ax1 = fig.add_subplot(gs[0, :2])

commodity_variance = df_all.groupby('commodity')['price'].var().sort_values(ascending=False)
top_n = min(5, len(commodity_variance))
top_commodities = commodity_variance.head(top_n).index

all_colors = ['#e74c3c', '#3498db', '#2ecc71', '#f39c12', '#9b59b6', '#e67e22', '#1abc9c']
attractive_colors = {comm: all_colors[i] for i, comm in enumerate(top_commodities)}

week_min = df_all['week'].min()
week_max = df_all['week'].max()

ax1.axvspan(week_min, -2, alpha=0.2, color='#27ae60', zorder=1)
ax1.text((week_min-2)/2, 0.97, 'ZONA AMAN',
         transform=ax1.get_xaxis_transform(),
         ha='center', va='top', fontsize=10, fontweight='bold',
         bbox=dict(boxstyle='round,pad=0.6', facecolor='#27ae60',
                  alpha=0.7, edgecolor='white', linewidth=2))

ax1.axvspan(-1, 1, alpha=0.2, color='#e74c3c', zorder=1)
ax1.text(0, 0.97, 'ZONA BAHAYA',
         transform=ax1.get_xaxis_transform(),
         ha='center', va='top', fontsize=10, fontweight='bold',
         bbox=dict(boxstyle='round,pad=0.6', facecolor='#e74c3c',
                  alpha=0.7, edgecolor='white', linewidth=2))

if week_max > 1:
    ax1.axvspan(2, week_max, alpha=0.2, color='#3498db', zorder=1)
    ax1.text((2+week_max)/2, 0.97, 'ZONA PEMULIHAN',
             transform=ax1.get_xaxis_transform(),
             ha='center', va='top', fontsize=10, fontweight='bold',
             bbox=dict(boxstyle='round,pad=0.6', facecolor='#3498db',
                      alpha=0.7, edgecolor='white', linewidth=2))

for commodity in top_commodities:
    mask = df_all['commodity'] == commodity
    comm_data = df_all[mask].groupby('week')['price'].mean().reset_index()

    color = attractive_colors[commodity]

    ax1.plot(comm_data['week'], comm_data['price'],
             linewidth=6, color=color, alpha=0.15, zorder=2)

    ax1.plot(comm_data['week'], comm_data['price'],
             marker='o', linewidth=2.5, markersize=7,
             color=color, label=commodity,
             markerfacecolor=color,
             markeredgecolor='white', markeredgewidth=1.5,
             alpha=0.9, zorder=3)

ax1.axvline(x=0, color='#2c3e50', linestyle='--', linewidth=3,
            alpha=0.7, zorder=4, label='Hari Event')

ax1.set_xlabel('Minggu (relatif terhadap event)', fontsize=11, fontweight='bold')
ax1.set_ylabel('Harga (Rp)', fontsize=11, fontweight='bold')

title_text = f'1. RECURRENCE RELATIONS: Tren Harga Top {top_n} Komoditas Volatile\nP(n+1) = P(n) × (1 + r)'
ax1.set_title(title_text, fontsize=12, fontweight='bold', pad=15)

legend = ax1.legend(fontsize=9, loc='upper left',
                   framealpha=0.5,
                   edgecolor='gray',
                   fancybox=True)
legend.get_frame().set_linewidth(1)
legend.get_frame().set_facecolor('white')

ax1.grid(True, alpha=0.2, linestyle='--', linewidth=0.8)

for spine in ax1.spines.values():
    spine.set_edgecolor('#2c3e50')
    spine.set_linewidth(2)

ax2 = fig.add_subplot(gs[0, 2])

aggregate_state_counts = df_all['state'].value_counts()
aggregate_state_counts = aggregate_state_counts.reindex(
    ['Normal', 'Siaga', 'Peringatan', 'PUNCAK'], fill_value=0
)

state_colors_pie = {'Normal': '#2ecc71', 'Siaga': '#f39c12',
                    'Peringatan': '#e67e22', 'PUNCAK': '#e74c3c'}
colors = [state_colors_pie[s] for s in aggregate_state_counts.index]

wedges, texts, autotexts = ax2.pie(aggregate_state_counts.values,
                                   labels=aggregate_state_counts.index,
                                   autopct='%1.1f%%', colors=colors,
                                   startangle=90,
                                   textprops={'fontsize': 9, 'fontweight': 'bold'})
for autotext in autotexts:
    autotext.set_color('white')
    autotext.set_fontweight('bold')

ax2.set_title(f'2. STRUKTUR DISKRIT\n(Distribusi FSM)',
              fontsize=11, fontweight='bold', pad=12)

ax3 = fig.add_subplot(gs[1, 0])

ax3.text(0.5, 0.6, 'INDUKSI\nMATEMATIS', ha='center', va='center',
        fontsize=14, fontweight='bold', transform=ax3.transAxes)
ax3.text(0.5, 0.4, 'Formula Terverifikasi\nP(n) = P(0) × ∏(1+rᵢ)',
        ha='center', va='center',
        fontsize=10, transform=ax3.transAxes)
ax3.text(0.5, 0.2, '✓ Basis (n=0)\n✓ Induktif (n→n+1)',
        ha='center', va='center',
        fontsize=9, color='green', fontweight='bold',
        transform=ax3.transAxes)
ax3.set_title(f'3. INDUKSI MATEMATIS\nPembuktian Formal',
              fontsize=11, fontweight='bold', pad=12)
ax3.axis('off')

ax4 = fig.add_subplot(gs[1, 1])

pos = {
    'Normal': (0, 0),
    'Siaga': (1.5, 0.5),
    'Peringatan': (3, 1),
    'PUNCAK': (4.5, 1.5)
}

nx.draw_networkx_nodes(G, pos,
                      node_color=[state_colors_pie.get(s, 'gray') for s in G.nodes()],
                      node_size=2500, alpha=0.95, ax=ax4,
                      edgecolors='black', linewidths=2)

edge_weights = [G[u][v]['weight'] * 5 for u, v in G.edges()]
nx.draw_networkx_edges(G, pos, width=edge_weights, alpha=0.6,
                      edge_color='gray', arrows=True, arrowsize=20,
                      arrowstyle='-|>', connectionstyle='arc3,rad=0.15', ax=ax4)

nx.draw_networkx_labels(G, pos, font_size=11, font_weight='bold', ax=ax4)

edge_labels = {(u, v): f"{G[u][v]['weight']:.2f}" for u, v in G.edges()}
nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=8, ax=ax4)

if 'shortest_path' in locals() and len(shortest_path) > 0:
    path_edges = [(shortest_path[i], shortest_path[i+1])
                  for i in range(len(shortest_path)-1)]
    nx.draw_networkx_edges(G, pos, edgelist=path_edges, width=6, alpha=0.9,
                          edge_color='red', arrows=True, arrowsize=25,
                          arrowstyle='-|>', ax=ax4)

ax4.set_xlim(-0.5, 5.5)
ax4.set_ylim(-0.5, 2.0)

ax4.set_title('4. TEORI GRAF\n(Merah = Shortest Path)',
              fontsize=11, fontweight='bold', pad=12)
ax4.axis('off')

ax5 = fig.add_subplot(gs[1, 2])

logic_data = {}
logic_data['KRITIS'] = len(df_all[df_all['state'].isin(['PUNCAK', 'Peringatan'])])
logic_data['AMAN_BELI'] = len(df_all[(df_all['state'] == 'Normal') & (df_all['week'] < -1)])
logic_data['HINDARI'] = len(df_all[(df_all['state'].isin(['PUNCAK', 'Peringatan'])) & (df_all['week'].abs() <= 1)])

colors_logic = ['red', 'green', 'darkred']
bars = ax5.barh(list(logic_data.keys()), list(logic_data.values()),
                color=colors_logic, alpha=0.8, edgecolor='black', linewidth=1.5)

ax5.set_xlabel('Jumlah', fontsize=10, fontweight='bold')
ax5.set_title('5. BOOLEAN LOGIC\nAturan Keputusan (∧, ∨, ¬)',
              fontsize=11, fontweight='bold', pad=12)
ax5.grid(True, alpha=0.3, axis='x')

for i, (key, val) in enumerate(logic_data.items()):
    pct = val / len(df_all) * 100
    ax5.text(val + max(logic_data.values())*0.02, i,
            f'{val} ({pct:.1f}%)', va='center', fontsize=9, fontweight='bold')

ax6 = fig.add_subplot(gs[2, 0])

k_vals = list(range(1, n_factors + 1))
comb_vals = [comb(n_factors, k) for k in k_vals]

bars = ax6.bar(k_vals, comb_vals, color='steelblue', alpha=0.8,
              edgecolor='black', linewidth=2, width=0.6)
ax6.set_xlabel('k (Jumlah Faktor)', fontsize=10, fontweight='bold')
ax6.set_ylabel('C(4,k)', fontsize=10, fontweight='bold')
ax6.set_title('6. KOMBINATORIKA\nC(n,k) = n!/(k!(n-k)!)',
              fontsize=11, fontweight='bold', pad=12)
ax6.set_xticks(k_vals)
ax6.grid(True, alpha=0.3, axis='y')

for k, v in zip(k_vals, comb_vals):
    ax6.text(k, v + 0.3, str(v), ha='center', fontweight='bold', fontsize=11)

ax7 = fig.add_subplot(gs[2, 1])

number_metrics = {}

if 'overall_gcd' in locals() and overall_gcd > 1:
    number_metrics['GCD Siklus'] = overall_gcd

if 'overall_lcm' in locals() and overall_lcm > 0 and overall_lcm < 100:
    number_metrics['LCM Sinkron'] = overall_lcm

if 'divisibility_stats' in locals() and len(divisibility_stats) > 0:
    max_div = max(divisibility_stats.items(), key=lambda x: x[1]['percentage'])
    number_metrics[f'Div Rp{max_div[0]}'] = max_div[1]['percentage']

if len(number_metrics) > 0:
    bars = ax7.barh(list(number_metrics.keys()), list(number_metrics.values()),
                    color='purple', alpha=0.8, edgecolor='black', linewidth=1.5)
    ax7.set_xlabel('Nilai', fontsize=10, fontweight='bold')
    ax7.set_title('7. TEORI BILANGAN\nGCD, LCM & Divisibilitas',
                  fontsize=11, fontweight='bold', pad=12)
    ax7.grid(True, alpha=0.3, axis='x')

    for i, (key, val) in enumerate(number_metrics.items()):
        ax7.text(val + max(number_metrics.values())*0.03, i,
                f'{val:.1f}', va='center', fontweight='bold', fontsize=10)
else:
    ax7.text(0.5, 0.5, 'Teori Bilangan\nGCD=1\nTidak Ada Siklus',
            ha='center', va='center',
            fontsize=10, fontweight='bold', transform=ax7.transAxes)
    ax7.axis('off')

ax8 = fig.add_subplot(gs[2, 2])

set_data = {
    'Selalu\nSpike (∩)': len(always_spike),
    'Kadang\nSpike (∪)': len(sometimes_spike),
    'Tidak\nSpike (ᶜ)': len(never_spike)
}

colors_sets = ['red', 'orange', 'green']
bars = ax8.bar(range(len(set_data)), list(set_data.values()),
              color=colors_sets, alpha=0.8, edgecolor='black', linewidth=1.5)

ax8.set_xticks(range(len(set_data)))
ax8.set_xticklabels(list(set_data.keys()), fontsize=9, fontweight='bold')
ax8.set_ylabel('Jumlah', fontsize=10, fontweight='bold')
ax8.set_title('8. TEORI HIMPUNAN\n∩, ∪, Complement',
              fontsize=11, fontweight='bold', pad=12)
ax8.grid(True, alpha=0.3, axis='y')

for i, val in enumerate(set_data.values()):
    ax8.text(i, val + 0.5, str(val), ha='center',
            fontweight='bold', fontsize=11)

ax9 = fig.add_subplot(gs[3, 0])

if 'pmf' in locals():
    prob_dist_plot = pmf
else:
    increase_bins = pd.cut(df_all['increase_pct'], bins=5)
    prob_dist_plot = increase_bins.value_counts(normalize=True).sort_index()

bars = ax9.bar(range(len(prob_dist_plot)), prob_dist_plot.values,
              color='coral', alpha=0.8, edgecolor='black', linewidth=1.5)
ax9.set_xticks(range(len(prob_dist_plot)))
ax9.set_xticklabels(prob_dist_plot.index, rotation=45, fontsize=8, ha='right')
ax9.set_ylabel('P(X=xᵢ)', fontsize=10, fontweight='bold')
ax9.set_title(f'9. PROBABILITAS DISKRIT\nE(X)={expected_value:.1f}% (PMF)',
              fontsize=11, fontweight='bold', pad=12)
ax9.grid(True, alpha=0.3, axis='y')

for i, prob in enumerate(prob_dist_plot.values):
    ax9.text(i, prob + 0.01, f'{prob:.2f}', ha='center',
            fontweight='bold', fontsize=8)

ax10 = fig.add_subplot(gs[3, 1])

P2 = np.linalg.matrix_power(P, 2)
df_P2 = pd.DataFrame(P2, index=states, columns=states)

sns.heatmap(df_P2, annot=True, fmt='.3f', cmap='YlOrRd', ax=ax10,
            cbar_kws={'label': 'P²', 'shrink': 0.8},
            linewidths=2, linecolor='black',
            annot_kws={'fontsize': 9, 'fontweight': 'bold'})
ax10.set_title('10. MARKOV CHAINS\nTransisi 2-Step (P²)',
              fontsize=11, fontweight='bold', pad=12)
ax10.set_xlabel('State Berikutnya', fontsize=9, fontweight='bold')
ax10.set_ylabel('State Saat Ini', fontsize=9, fontweight='bold')
ax10.tick_params(labelsize=8)

ax11 = fig.add_subplot(gs[3, 2])

truth_data = []

conditions = [
    ('Normal', 'Awal', False, False),
    ('Normal', 'Dekat', False, True),
    ('Siaga', 'Awal', False, False),
    ('Siaga', 'Dekat', True, True),
    ('Peringatan', 'Awal', True, False),
    ('Peringatan', 'Dekat', True, True),
    ('PUNCAK', 'Awal', True, False),
    ('PUNCAK', 'Dekat', True, True),
]

for state, timing, is_critical, is_avoid in conditions:
    is_safe = (state == 'Normal' and timing == 'Awal')
    truth_data.append({
        'Kondisi': f'{state}\n{timing}',
        'KRITIS': 1 if is_critical else 0,
        'AMAN_BELI': 1 if is_safe else 0,
        'HINDARI': 1 if is_avoid else 0
    })

df_truth = pd.DataFrame(truth_data)
df_truth_matrix = df_truth.set_index('Kondisi')[['KRITIS', 'AMAN_BELI', 'HINDARI']]

sns.heatmap(df_truth_matrix, annot=True, fmt='d', cmap='RdYlGn_r', ax=ax11,
            cbar_kws={'label': 'Boolean', 'shrink': 0.8},
            linewidths=2, linecolor='black',
            annot_kws={'fontsize': 10, 'fontweight': 'bold'},
            vmin=0, vmax=1)
ax11.set_title('11. TRUTH TABLE\nAturan Keputusan Boolean',
              fontsize=11, fontweight='bold', pad=12)
ax11.set_xlabel('Output Keputusan', fontsize=9, fontweight='bold')
ax11.set_ylabel('State & Timing', fontsize=9, fontweight='bold')
ax11.tick_params(labelsize=8)

plt.tight_layout()
plt.show()

pause_section()

In [ ]:
# ===================================================================
# SECTION 11: KESIMPULAN & REKOMENDASI
# ===================================================================

print("\n" + "="*90)
print(" 11. KESIMPULAN & REKOMENDASI ".center(90))
print("="*90)

print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║          RINGKASAN: 9 KONSEP MATEMATIKA DISKRIT YANG DIGUNAKAN               ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")

if SHOW_EXAMPLE:
    print(f"📊 MODE ANALISIS: SEMUA KOMODITAS")
    print(f"   Total komoditas dianalisis: {len(commodities_list)}")
else:
    print(f"📊 MODE ANALISIS: KATEGORI {COMMODITY_GROUPS[kategori_choice]['name'].upper()}")
    print(f"   Komoditas dalam kategori: {', '.join(commodities_list)}")

print()

before_event = df_all[df_all['week'] < 0]
after_event = df_all[df_all['week'] > 0]
at_event = df_all[df_all['week'] == 0]

avg_increase_before = before_event['increase_pct'].mean() if len(before_event) > 0 else 0
avg_increase_after = after_event['increase_pct'].mean() if len(after_event) > 0 else 0
avg_increase_at = at_event['increase_pct'].mean() if len(at_event) > 0 else 0

most_common_state = df_all['state'].mode()[0]

if 'expected_value' not in locals():
    expected_value = df_all['increase_pct'].mean()

print(f"1️⃣ FINITE STATE MACHINE (FSM)")
print(f"     Q = {{Normal, Siaga, Peringatan, PUNCAK}}")
print(f"     Total observasi: {len(df_all)}, State dominan: {most_common_state}")

print(f"\n2️⃣ RECURRENCE RELATIONS")
print(f"     Formula: P(n+1) = P(n) × (1 + r)")
print(f"     Rata-rata kenaikan: {avg_increase_before:+.2f}% (sebelum), {avg_increase_at:+.2f}% (saat event)")

print(f"\n3️⃣ INDUKSI MATEMATIS")
print(f"     Formula P(n) = P(0) × ∏(1+rᵢ) terbukti untuk semua n ∈ ℕ")
print(f"     Basis (n=0) dan step induktif (n→n+1) terverifikasi")

print(f"\n4️⃣ TEORI GRAF (GRAPH THEORY)")
print(f"     G = (V, E): |V|={G.number_of_nodes()}, |E|={G.number_of_edges()}, density={nx.density(G):.3f}")
if 'shortest_path' in locals() and len(shortest_path) > 0:
    print(f"     Shortest path Normal → PUNCAK: {' → '.join(shortest_path)} ({len(shortest_path)-1} transisi)")
else:
    print(f"     Graf transisi state berhasil dibangun")

print(f"\n5️⃣ BOOLEAN LOGIC (OPERASI LOGIKA)")
print(f"     Operator: ∧, ∨, ¬, → untuk aturan keputusan")
if 'KRITIS' in df_all.columns:
    print(f"     KRITIS: {df_all['KRITIS'].sum()}, AMAN_BELI: {df_all['AMAN_BELI'].sum()}, HINDARI: {df_all['HINDARI'].sum()}")
else:
    print(f"     Aturan logika berhasil diterapkan")

print(f"\n6️⃣ KOMBINATORIKA (COMBINATORICS)")
if 'total_combinations' in locals():
    print(f"     Total kombinasi faktor: 2^{n_factors} - 1 = {total_combinations}")
else:
    print(f"     Kombinasi C(n,k) untuk {n_factors} faktor dianalisis")

print(f"\n7️⃣ TEORI BILANGAN (NUMBER THEORY)")
if 'overall_gcd' in locals() and overall_gcd > 1:
    print(f"     GCD = {overall_gcd} (siklus {overall_gcd}-minggu terdeteksi)")
    if 'overall_lcm' in locals():
        print(f"     LCM = {overall_lcm}")
else:
    print(f"     GCD = 1 (tidak ada siklus umum antar komoditas)")
print(f"     Modular arithmetic & divisibilitas dianalisis")

print(f"\n8️⃣ TEORI HIMPUNAN (SET THEORY)")
print(f"     |U| = {len(commodities_list)}, |∩| = {len(always_spike)}, |∪| = {len(sometimes_spike)}, |ᶜ| = {len(never_spike)}")
print(f"     Operasi ∩, ∪, -, ᶜ berhasil diaplikasikan")

print(f"\n9️⃣ TEORI PELUANG DISKRIT (DISCRETE PROBABILITY)")
print(f"     PMF dihitung untuk ruang sampel diskrit")
print(f"     E(X) = {expected_value:.2f}%")
if 'Normal' in states and 'PUNCAK' in states:
    normal_idx = states.index('Normal')
    peak_idx = states.index('PUNCAK')
    P2 = np.linalg.matrix_power(P, 2)
    print(f"     Markov: P²(Normal→PUNCAK) = {P2[normal_idx, peak_idx]:.4f}")

print(f"""

╔══════════════════════════════════════════════════════════════════════════════╗
║                       REKOMENDASI STRATEGIS                                  ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")

week_min = df_all['week'].min()
week_max = df_all['week'].max()

print(f"📅 TIMELINE PEMBELIAN:")
print(f"   🟢 Minggu {week_min:.0f} s/d -2: OPTIMAL (BELI)")
print(f"      State dominan: Normal, rata-rata kenaikan: {avg_increase_before:+.2f}%")

print(f"\n   🔴 Minggu -1 s/d +1: HINDARI")
print(f"      State krisis (Peringatan/PUNCAK), rata-rata kenaikan: {avg_increase_at:+.2f}%")

if week_max > 1:
    print(f"\n   🔵 Minggu +2 s/d +{week_max:.0f}: PEMULIHAN")
    print(f"      Harga mulai stabil, rata-rata kenaikan: {avg_increase_after:+.2f}%")

print(f"\n🎯 PRIORITAS MONITORING:")
print(f"   (Berdasarkan Coefficient of Variation - volatilitas relatif)")

df_category = df_all[df_all['commodity'].isin(commodities_list)]
commodity_risk = df_category.groupby('commodity')['increase_pct'].agg(['mean', 'std'])
commodity_risk['cv'] = abs(commodity_risk['std'] / (commodity_risk['mean'] + 0.001))  # Avoid division by zero
commodity_risk = commodity_risk.sort_values('cv', ascending=False)

top_n_display = min(5, len(commodity_risk)) if SHOW_EXAMPLE else len(commodity_risk)

for i, (commodity, row) in enumerate(commodity_risk.head(top_n_display).iterrows(), 1):
    cv = row['cv']
    risk = 'TINGGI' if cv > 1.0 else 'SEDANG' if cv > 0.5 else 'RENDAH'
    emoji = '🔴' if risk == 'TINGGI' else '🟡' if risk == 'SEDANG' else '🟢'
    print(f"   {i}. {emoji} {commodity:20s} - {risk:7s} (CV={cv:.2f}, Δ rata-rata={row['mean']:+.1f}%)")

print(f"\n💡 INSIGHT UTAMA:")

if SHOW_EXAMPLE:
    print(f"   • Analisis komprehensif {len(commodities_list)} komoditas sembako")
else:
    print(f"   • Analisis fokus pada kategori {COMMODITY_GROUPS[kategori_choice]['name']}")

print(f"   • State Normal mendominasi ({(df_all['state']=='Normal').sum()/len(df_all)*100:.1f}% observasi)")
print(f"   • Kenaikan tertinggi terjadi di minggu 0 (event): {avg_increase_at:+.1f}%")
print(f"   • {len(always_spike)} komoditas selalu spike, {len(never_spike)} selalu stabil")

if 'overall_gcd' in locals() and overall_gcd > 1:
    print(f"   • Siklus {overall_gcd}-minggu terdeteksi dalam timing peak")
else:
    print(f"   • Tidak ada siklus umum (setiap komoditas punya pola berbeda)")

if not SHOW_EXAMPLE:
    cat_name = COMMODITY_GROUPS[kategori_choice]['name']
    avg_cat_increase = df_category['increase_pct'].mean()

    print(f"\n📌 REKOMENDASI SPESIFIK {cat_name.upper()}:")
    if avg_cat_increase > expected_value:
        print(f"   • Kategori ini LEBIH VOLATILE dari rata-rata global ({avg_cat_increase:.1f}% vs {expected_value:.1f}%)")
        print(f"   • Prioritaskan pembelian jauh sebelum event (minggu {week_min:.0f} s/d -3)")
    else:
        print(f"   • Kategori ini LEBIH STABIL dari rata-rata global ({avg_cat_increase:.1f}% vs {expected_value:.1f}%)")
        print(f"   • Masih aman beli hingga minggu -2 sebelum event")

print(f"""

KESIMPULAN:
════════════════════════════════════════════════════════════════════════════════
9 konsep matematika diskrit berhasil diaplikasikan untuk memodelkan dinamika
harga sembako menjelang hari besar. Sebanyak {len(df_all)} observasi dianalisis
{"dari " + str(len(commodities_list)) + " komoditas" if SHOW_EXAMPLE else "untuk kategori " + COMMODITY_GROUPS[kategori_choice]['name'] + " (" + ", ".join(commodities_list) + ")"},
semua formula terverifikasi secara matematis dan empiris.

Rekomendasi: Beli sembako pada minggu {week_min:.0f} s/d -2 untuk penghematan
maksimal, dan hindari pembelian pada minggu -1 s/d +1 saat harga puncak.
════════════════════════════════════════════════════════════════════════════════
""")

pause_section()

In [ ]:
# ===================================================================
# SECTION 12: EXPORT REKOMENDASI
# ===================================================================

print("\n" + "="*90)
print(" 12. EXPORT REKOMENDASI ".center(90))
print("="*90)

print("""
File export: Panduan lengkap per komoditas
Format: CSV yang mudah dibaca di Excel/Google Sheets
""")

print("\nMempersiapkan analisis per komoditas...")

export_data = []

for commodity in commodities_list:
    comm_data = df_all[df_all['commodity'] == commodity].copy()

    if len(comm_data) > 0:
        avg_increase = comm_data['increase_pct'].mean()
        std_increase = comm_data['increase_pct'].std()

        percentile_95 = comm_data['increase_pct'].quantile(0.95)
        max_increase_raw = comm_data['increase_pct'].max()
        max_increase = min(max_increase_raw, percentile_95 * 1.5)

        if abs(avg_increase) > 1:
            cv = abs(std_increase / avg_increase)
        else:
            cv = std_increase / 10 if std_increase > 0 else 0.1

        peak_price = comm_data['price'].max()
        peak_week = comm_data[comm_data['price'] == peak_price]['week'].iloc[0]

        state_counts = comm_data['state'].value_counts()
        most_common_state = state_counts.idxmax() if len(state_counts) > 0 else 'Normal'
        crisis_count = state_counts.get('PUNCAK', 0) + state_counts.get('Peringatan', 0)
        crisis_pct = (crisis_count / len(comm_data) * 100)

        safe_zone = comm_data[comm_data['week'] <= -2]
        if len(safe_zone) >= 2:
            normal_price = safe_zone['price'].median()
        elif len(safe_zone) > 0:
            normal_price = safe_zone['price'].mean()
        else:
            normal_price = comm_data['price'].min()

        danger_zone = comm_data[(comm_data['week'] >= -1) & (comm_data['week'] <= 1)]
        if len(danger_zone) >= 2:
            danger_price = danger_zone['price'].quantile(0.75)
        elif len(danger_zone) > 0:
            danger_price = danger_zone['price'].max()
        else:
            danger_price = peak_price

        if danger_price > normal_price * 1.02:  # >2% increase
            saving_amount = danger_price - normal_price
            saving_pct = (saving_amount / danger_price * 100)
        else:
            saving_amount = 0
            saving_pct = 0

        risk_score = (abs(max_increase) * 0.6) + (cv * 20) + (crisis_pct * 0.4)

        has_real_increase = (danger_price > normal_price * 1.05)

        if has_real_increase and (risk_score > 70 or (abs(max_increase) > 80 and cv > 1.5)):
            risk_level = 'SANGAT TINGGI'
            urgency = 'PRIORITAS 1'
            timing = 'Beli 4+ minggu sebelum'
            stock = 'Stok 1-2 bulan'
        elif has_real_increase and (risk_score > 40 or (abs(max_increase) > 40 and cv > 0.8)):
            risk_level = 'TINGGI'
            urgency = 'PRIORITAS 2'
            timing = 'Beli 3-4 minggu sebelum'
            stock = 'Stok 3-4 minggu'
        elif has_real_increase and (risk_score > 20 or (abs(max_increase) > 15 and cv > 0.4)):
            risk_level = 'SEDANG'
            urgency = 'PRIORITAS 3'
            timing = 'Beli 2-3 minggu sebelum'
            stock = 'Stok 2-3 minggu'
        else:
            risk_level = 'RENDAH'
            urgency = 'PRIORITAS 4'
            timing = 'Timing fleksibel'
            stock = 'Stok normal 1-2 minggu'

        if crisis_pct > 40:
            status = 'SERING KRISIS'
        elif crisis_pct > 20:
            status = 'KADANG KRISIS'
        else:
            status = 'RELATIF STABIL'

        if saving_pct >= 10:
            hemat_text = f'Hemat {saving_pct:.1f}% (~Rp {saving_amount:,.0f})'
        elif saving_pct >= 2:
            hemat_text = f'Hemat {saving_pct:.1f}%'
        elif saving_pct >= 0.1:
            hemat_text = f'Hemat minimal (Rp {saving_amount:,.0f})'
        else:
            hemat_text = 'Harga stabil sepanjang periode'

        export_data.append({
            'Komoditas': commodity,
            'Tingkat_Risiko': risk_level,
            'Prioritas': urgency,
            'Kapan_Beli': timing,
            'Jumlah_Stok': stock,
            'Status': status,
            'Harga_Normal': f'Rp {normal_price:,.0f}',
            'Harga_Puncak': f'Rp {danger_price:,.0f}',
            'Selisih': f'Rp {saving_amount:,.0f}',
            'Potensi_Hemat': f'{saving_pct:.1f}%',
            'Kenaikan_Rata': f'{avg_increase:.1f}%',
            'Kenaikan_Maks': f'{max_increase:.1f}%',
            'Minggu_Puncak': f'H{peak_week:+.0f}',
            'Volatilitas_CV': f'{cv:.2f}',
            'Risk_Score': f'{risk_score:.1f}',
            'Rekomendasi': f'{timing}. {stock}. {hemat_text}.'
        })

df_export = pd.DataFrame(export_data)

commodity_order = [
    'Beras', 'Gula Pasir', 'Minyak Goreng',
    'Daging Sapi', 'Daging Ayam', 'Telur Ayam',
    'Bawang Merah', 'Bawang Putih', 'Cabai Merah', 'Cabai Rawit'
]

df_export['Komoditas'] = pd.Categorical(
    df_export['Komoditas'],
    categories=commodity_order,
    ordered=True
)

df_export_final = df_export.sort_values('Komoditas').reset_index(drop=True)

filename = 'rekomendasi_per_komoditas.csv'
df_export_final.to_csv(filename, index=False, encoding='utf-8-sig')

print(f"\n✅ File berhasil dibuat: {filename}")
print(f"   Total komoditas: {len(df_export_final)}")

print(f"\n📊 DISTRIBUSI TINGKAT RISIKO:")
for risk in ['SANGAT TINGGI', 'TINGGI', 'SEDANG', 'RENDAH']:
    count = len(df_export_final[df_export_final['Tingkat_Risiko'] == risk])
    if count > 0:
        items = df_export_final[df_export_final['Tingkat_Risiko'] == risk]['Komoditas'].tolist()
        bar = '█' * count
        print(f"  {risk:15s} {bar} ({count}) - {', '.join(items)}")

print(f"\n📋 PREVIEW FILE:")
print("="*80)
for idx, row in df_export_final.iterrows():
    risk_icon = '🔴' if row['Tingkat_Risiko'] == 'SANGAT TINGGI' else \
                '🟠' if row['Tingkat_Risiko'] == 'TINGGI' else \
                '🟡' if row['Tingkat_Risiko'] == 'SEDANG' else '🟢'
    print(f"{idx+1:2d}. {risk_icon} {row['Komoditas']:15s} - {row['Tingkat_Risiko']:15s} "
          f"({row['Prioritas']}) - Hemat: {row['Potensi_Hemat']}")

from google.colab import files
files.download(filename)

print(f"""

CARA MENGGUNAKAN:
─────────────────────────────────────────────────────────────
1. Buka di Excel/Google Sheets
2. Sort berdasarkan "Prioritas" untuk urgent items
3. Fokus pada "Potensi_Hemat" (lebih reliable dari Kenaikan_Rata)
4. "Tingkat_Risiko" mempertimbangkan volatilitas & kenaikan riil

═══════════════════════════════════════════════════════════════════════════════
""")

print("\n✅ File telah diekspor!")

pause_section()


print("\n" + "="*90)
print(" ANALISIS MATEMATIKA DISKRIT SELESAI ".center(90))
print("="*90)

print(f"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                           RINGKASAN ANALISIS                                 ║
╚══════════════════════════════════════════════════════════════════════════════╝

Total Section: 12 Section
Total Visualisasi: 11 grafik dalam dashboard
Total File Export: 1 file CSV (baseline konsisten)

DATASET:
  • Total Observasi: {len(df_all)}
  • Jumlah Komoditas: {len(commodities_list)}{"" if SHOW_EXAMPLE else f" (Kategori: {COMMODITY_GROUPS[kategori_choice]['name']})"}
  • Jumlah Event: {len(events)}
  • Periode: Minggu {df_all['week'].min():.0f} s/d {df_all['week'].max():.0f}

9 KONSEP MATEMATIKA DISKRIT:
  1. ✓ Finite State Machine (FSM)
  2. ✓ Recurrence Relations
  3. ✓ Induksi Matematis
  4. ✓ Teori Graf
  5. ✓ Boolean Logic
  6. ✓ Kombinatorika
  7. ✓ Teori Bilangan
  8. ✓ Teori Himpunan
  9. ✓ Teori Peluang Diskrit + Markov Chains

DELIVERABLES:
  ✓ 12 Section analisis lengkap
  ✓ 11 visualisasi komprehensif
  ✓ 1 file CSV rekomendasi (baseline konsisten)
  ✓ Semua formula terverifikasi

╔══════════════════════════════════════════════════════════════════════════════╗
║                        TERIMA KASIH                                          ║
╚══════════════════════════════════════════════════════════════════════════════╝

Proyek Matematika Diskrit telah selesai dengan sukses!
Semua konsep berhasil diaplikasikan dengan metodologi yang konsisten.

═══════════════════════════════════════════════════════════════════════════════
""")